# Evolutionary Computation Assignment 3

- Krzysztof Szala 144571
- Vadym Repetskyi 155610


In [1]:
from utils import TspInstance, random_solution, weighted_regret
import numpy as np
import pandas as pd
import itertools

In [2]:
def intra_route_move(tsp, solution, intra_type, steepest):
    solution = solution.copy()
    indexes = range(len(solution))
    neighborhood = [(u, v) for u in indexes for v in indexes if u != v]
    np.random.shuffle(neighborhood)

    min_cost = 0
    selected_a = selected_b = None

    for index_a, index_b in neighborhood:
        # we can draw first edge with any node, but second have to start from not node_a and not node_a+1 or node_a-1
        if intra_type != "nodes" and abs(index_a - index_b) == 1:
            continue

        a = solution[index_a]
        b = solution[index_b]

        a_prev = solution[index_a - 1]
        b_prev = solution[index_b - 1]
        if index_a - 1 < 0:
            a_prev = solution[len(solution) - 1]
        if index_b - 1 < 0:
            b_prev = solution[len(solution) - 1]

        a_next = solution[(index_a + 1) % len(solution)]
        b_next = solution[(index_b + 1) % len(solution)]
        
        a_next_next = solution[(index_a + 2) % len(solution)]
        b_next_next = solution[(index_b + 2) % len(solution)]

        if intra_type == "nodes":
            if abs(index_a - index_b) == 1: # Adjacent case
                if index_a < index_b:  # Case where a is directly before b
                    cost_change = (
                        tsp.distance_matrix[a_prev, b]
                        + tsp.distance_matrix[a, b_next]
                        - tsp.distance_matrix[a_prev, a]
                        - tsp.distance_matrix[b, b_next]
                    )
                else:  # Case where b is directly before a
                    cost_change = (
                        tsp.distance_matrix[b_prev, a]
                        + tsp.distance_matrix[b, a_next]
                        - tsp.distance_matrix[b_prev, b]
                        - tsp.distance_matrix[a, a_next]
                    )
            elif abs(index_a - index_b) == len(solution) - 1:  # Adjacent case - last and first node
                if index_a > index_b:
                    cost_change = (
                        tsp.distance_matrix[a_prev, b]
                        + tsp.distance_matrix[a, b_next]
                        - tsp.distance_matrix[a_prev, a]
                        - tsp.distance_matrix[b, b_next]
                    )
                else:
                    cost_change = (
                        tsp.distance_matrix[b_prev, a]
                        + tsp.distance_matrix[b, a_next]
                        - tsp.distance_matrix[b_prev, b]
                        - tsp.distance_matrix[a, a_next]
                    )
            else:
                # Non-adjacent case, use the general formula
                cost_change = (
                    tsp.distance_matrix[a_prev, b]
                    + tsp.distance_matrix[b, a_next]
                    + tsp.distance_matrix[b_prev, a]
                    + tsp.distance_matrix[a, b_next]
                    - tsp.distance_matrix[a_prev, a]
                    - tsp.distance_matrix[a, a_next]
                    - tsp.distance_matrix[b_prev, b]
                    - tsp.distance_matrix[b, b_next]
                )

        else:
            cost_change = (
                tsp.distance_matrix[a, b]
                + tsp.distance_matrix[a_next, b_next]
                + tsp.distance_matrix[b, a_next_next]
                + tsp.distance_matrix[a_next, b_prev]
                - tsp.distance_matrix[a, a_next]
                - tsp.distance_matrix[b, b_next]
            )

        if cost_change < min_cost:
            min_cost = cost_change
            selected_a = index_a
            selected_b = index_b

            if not steepest:
                break

    if selected_a is not None:
        if intra_type == "nodes":
            # Just swap the two nodes

            solution[selected_a], solution[selected_b] = (
                solution[selected_b],
                solution[selected_a],
            )
        else:
            # node_a stays at the same position
            # node_a+1 become node_b
            # node_b become node_a+1
            # node b+1 stays at the same position

            solution[(selected_a + 1) % len(solution)], solution[selected_b] = (
                solution[selected_b],
                solution[(selected_a + 1) % len(solution)],
            )

    return solution, min_cost


def inter_route_move(tsp, solution, steepest):
    solution = solution.copy()
    possible_unselect = set(solution)
    possible_select = set(range(tsp.size)) - possible_unselect
    neighborhood = list(itertools.product(possible_unselect, possible_select))
    np.random.shuffle(neighborhood)

    min_cost = 0
    to_unselect = to_select = None

    for test_unselect, test_select in neighborhood:
        insert_index = np.where(solution == test_unselect)[0][0]

        prev = solution[insert_index - 1]
        next = solution[(insert_index + 1) % len(solution)]

        cost_change = (
            tsp.node_costs[test_select]
            + tsp.distance_matrix[prev, test_select]
            + tsp.distance_matrix[test_select, next]
            - tsp.node_costs[test_unselect]
            - tsp.distance_matrix[prev, test_unselect]
            - tsp.distance_matrix[test_unselect, next]
        )

        if cost_change < min_cost:
            min_cost = cost_change
            to_unselect = test_unselect
            to_select = test_select

            if not steepest:
                break

    if to_unselect is not None and not steepest:
        solution[np.where(solution == to_unselect)] = to_select

    if steepest:
        return min_cost, to_unselect, to_select
    else:
        return solution, min_cost

## Local Search

### Greedy

1. Choose arbitrarily starting node and generate random/weighted(best heuristic so far) solution
2. Randomly select move from your neighbourhood
3. Iterate over all possible moves, until you find first which decrease cost (cost change would be negative) or if you would check all possible moves.
4. Repeat 2 and 3 step as long as you get a move which decrease overall solution cost, when you don't find such move return solution.

Randomly selecting solution

1. Randomly choose if inter/intra route move (if it's greedy, for steepest it doesn't matter)
2. Randomly look for next solution, remembering already visited solutions (after visiting all intra/inter moves without getting cost change decrease, check the other all possible moves within inter/intra space)

### Steepest

1. Choose arbitrarily starting node and generate random/weighted(best heuristic so far) solution
2. Check all possible moves from your neighbourhood and choose a move with the biggest decrease of cost.
3. Repeat 2 and 3 step as long as you get a move which decrease overall solution cost, when you don't find such move return solution.

In [3]:
def local_search(
    tsp: TspInstance,
    start_node: int,
    initial_solution_getter: callable,
    intra_type: str,
    steepest: bool,
):
    solution = initial_solution_getter(tsp, start_node)

    previous = list()

    debug = False
    if debug:
        counter = 0
        last_sol = None
    
    while True:
        if debug:
            counter += 1
            last_sol = solution.copy()
            previous.append((str(solution), counter))
        
        if steepest:
            cost_change_1, to_unselect, to_select = inter_route_move(tsp, solution, steepest)
            solution_2, cost_change_2 = intra_route_move(
                tsp, solution, intra_type, steepest
            )
            if cost_change_1 < cost_change_2:
                cost_change = cost_change_1
                if to_unselect is not None:
                    solution[np.where(solution == to_unselect)] = to_select
            else:
                solution, cost_change = solution_2, cost_change_2
        else:
            if np.random.rand() < 0.5:
                solution, cost_change = inter_route_move(tsp, solution, steepest)

                if cost_change == 0:
                    solution, cost_change = intra_route_move(
                        tsp, solution, intra_type, steepest
                    )
            else:
                solution, cost_change = intra_route_move(
                    tsp, solution, intra_type, steepest
                )

                if cost_change == 0:
                    solution, cost_change = inter_route_move(tsp, solution, steepest)

        if cost_change == 0:
            break

        if debug:
            if str(solution) in [x[0] for x in previous]:
                # previous solution

                last_cost = tsp.get_cost(last_sol)
                current_cost = tsp.get_cost(solution)
                
                if last_cost < current_cost:
                
                    print("Previous solution")
                    print(str(previous[-1][0]))
                    list_previous = [x[1] for x in previous if x[0] == str(solution)]
                    print("Current solution")
                    print(str(solution))
                    
                    print("Last cost", last_cost)
                    print("Current cost", current_cost)
                    print("Already seen solution", "Current cost_change: ", cost_change, " in previous iteration ", list_previous)

    return solution

## Experiments


In [4]:
columns = []
experiments = []

for instance in (TspInstance("TSPA.csv"), TspInstance("TSPB.csv")):
    for intra_type in ("nodes", "edges"):
        for steepest in (True, False):
            for initial_solution_getter in (random_solution, weighted_regret):
                column = f"{instance.file_path[3]} {"steepest" if steepest else "greedy"} {intra_type} {initial_solution_getter.__name__.split("_")[0]}".upper()
                print(column)
                columns.append(column)
                experiments.append(
                    instance.run_experiments(
                        local_search, initial_solution_getter, intra_type, steepest
                    )
                )

A STEEPEST NODES RANDOM
A STEEPEST NODES WEIGHTED
A GREEDY NODES RANDOM
A GREEDY NODES WEIGHTED
A STEEPEST EDGES RANDOM
A STEEPEST EDGES WEIGHTED
A GREEDY EDGES RANDOM
A GREEDY EDGES WEIGHTED
B STEEPEST NODES RANDOM
B STEEPEST NODES WEIGHTED
B GREEDY NODES RANDOM
B GREEDY NODES WEIGHTED
B STEEPEST EDGES RANDOM
B STEEPEST EDGES WEIGHTED
B GREEDY EDGES RANDOM
B GREEDY EDGES WEIGHTED


<div>
  <style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }

  </style>
  <table border="1" class="dataframe">
    <thead>
      <tr style="text-align: right;">
        <th></th>
        <th>Random A</th>
        <th>NN-End A</th>
        <th>NN-Any A</th>
        <th>Cycle A</th>
        <th>Regret A</th>
        <th>Weighted A</th>
        <th>Random B</th>
        <th>NN-End B</th>
        <th>NN-Any B</th>
        <th>Cycle B</th>
        <th>Regret B</th>
        <th>Weighted B</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <th>min</th>
        <td>235530.0</td>
        <td>76880.0</td>
        <td>71255.0</td>
        <td>70785.0</td>
        <td>105692.0</td>
        <td>70154.0</td>
        <td>187468.0</td>
        <td>47940.0</td>
        <td>48973.0</td>
        <td>49038.0</td>
        <td>66162.0</td>
        <td>46832.0</td>
      </tr>
      <tr>
        <th>max</th>
        <td>294275.0</td>
        <td>105198.0</td>
        <td>74220.0</td>
        <td>76026.0</td>
        <td>126951.0</td>
        <td>73395.0</td>
        <td>239215.0</td>
        <td>67185.0</td>
        <td>57348.0</td>
        <td>57456.0</td>
        <td>78406.0</td>
        <td>55700.0</td>
      </tr>
      <tr>
        <th>avg</th>
        <td>264836.7</td>
        <td>83680.24</td>
        <td>72661.445</td>
        <td>73030.805</td>
        <td>115164.15</td>
        <td>71943.07</td>
        <td>213266.78</td>
        <td>52485.63</td>
        <td>51390.06</td>
        <td>51660.935</td>
        <td>72381.19</td>
        <td>50824.285</td>
      </tr>
    </tbody>
  </table>
</div>


In [ ]:
pd.DataFrame(
    np.array(tuple(map(lambda x: x[:-1], experiments))).T,
    columns=columns,
    index=(
        "min_cost",
        "max_cost",
        "average_cost",
        "min_time",
        "max_time",
        "average_time",
    ),
)

,A STEEPEST NODES RANDOM,A STEEPEST NODES WEIGHTED,A GREEDY NODES RANDOM,A GREEDY NODES WEIGHTED,A STEEPEST EDGES RANDOM,A STEEPEST EDGES WEIGHTED,A GREEDY EDGES RANDOM,A GREEDY EDGES WEIGHTED,B STEEPEST NODES RANDOM,B STEEPEST NODES WEIGHTED,B GREEDY NODES RANDOM,B GREEDY NODES WEIGHTED,B STEEPEST EDGES RANDOM,B STEEPEST EDGES WEIGHTED,B GREEDY EDGES RANDOM,B GREEDY EDGES WEIGHTED
min_cost,79543.000000,77166.000000,79716.000000,75599.000000,94636.000000,85296.000000,91560.000000,84341.000000,55802.000000,50324.000000,54581.000000,49701.000000,61922.000000,54470.000000,61712.000000,52449.000000
max_cost,98280.000000,90033.000000,94861.000000,91733.000000,122066.000000,103198.000000,111538.000000,101374.000000,72309.000000,60689.000000,70773.000000,60172.000000,85766.000000,69741.000000,82440.000000,68511.000000
average_cost,88041.030000,83777.340000,86056.665000,83129.250000,104366.425000,92531.985000,101610.135000,91750.625000,62880.750000,54201.230000,61111.105000,53922.790000,74009.900000,59236.900000,70794.630000,58892.755000
min_time,3.496069,1.994131,0.556165,0.855200,2.082852,1.449627,0.386965,0.598760,3.497686,1.907724,0.408789,0.614799,2.271810,1.450575,0.527983,0.596173
max_time,5.967110,5.577237,1.843326,1.813898,3.411806,3.010580,1.006590,1.052440,5.933967,3.427369,1.288425,1.326740,3.772325,2.763644,1.469050,1.042160
average_time,4.673874,2.841779,1.135089,1.200964,2.733647,1.989892,0.625281,0.777606,4.453023,2.512768,0.786179,0.941636,2.915474,2.001247,0.818187,0.757612


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>A STEEPEST NODES RANDOM</th>
      <th>A STEEPEST NODES WEIGHTED</th>
      <th>A GREEDY NODES RANDOM</th>
      <th>A GREEDY NODES WEIGHTED</th>
      <th>A STEEPEST EDGES RANDOM</th>
      <th>A STEEPEST EDGES WEIGHTED</th>
      <th>A GREEDY EDGES RANDOM</th>
      <th>A GREEDY EDGES WEIGHTED</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>min_cost</th>
      <td>79543.000000</td>
      <td>77166.000000</td>
      <td>79716.000000</td>
      <td>75599.000000</td>
      <td>94636.000000</td>
      <td>85296.000000</td>
      <td>91560.000000</td>
      <td>84341.000000</td>
    </tr>
    <tr>
      <th>max_cost</th>
      <td>98280.000000</td>
      <td>90033.000000</td>
      <td>94861.000000</td>
      <td>91733.000000</td>
      <td>122066.000000</td>
      <td>103198.000000</td>
      <td>111538.000000</td>
      <td>101374.000000</td>
    </tr>
    <tr>
      <th>average_cost</th>
      <td>88041.030000</td>
      <td>83777.340000</td>
      <td>86056.665000</td>
      <td>83129.250000</td>
      <td>104366.425000</td>
      <td>92531.985000</td>
      <td>101610.135000</td>
      <td>91750.625000</td>
    </tr>
    <tr>
      <th>min_time</th>
      <td>3.496069</td>
      <td>1.994131</td>
      <td>0.556165</td>
      <td>0.855200</td>
      <td>2.082852</td>
      <td>1.449627</td>
      <td>0.386965</td>
      <td>0.598760</td>
    </tr>
    <tr>
      <th>max_time</th>
      <td>5.967110</td>
      <td>5.577237</td>
      <td>1.843326</td>
      <td>1.813898</td>
      <td>3.411806</td>
      <td>3.010580</td>
      <td>1.006590</td>
      <td>1.052440</td>
    </tr>
    <tr>
      <th>average_time</th>
      <td>4.673874</td>
      <td>2.841779</td>
      <td>1.135089</td>
      <td>1.200964</td>
      <td>2.733647</td>
      <td>1.989892</td>
      <td>0.625281</td>
      <td>0.777606</td>
    </tr>
  </tbody>
</table>
</div>

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>B STEEPEST NODES RANDOM</th>
      <th>B STEEPEST NODES WEIGHTED</th>
      <th>B GREEDY NODES RANDOM</th>
      <th>B GREEDY NODES WEIGHTED</th>
      <th>B STEEPEST EDGES RANDOM</th>
      <th>B STEEPEST EDGES WEIGHTED</th>
      <th>B GREEDY EDGES RANDOM</th>
      <th>B GREEDY EDGES WEIGHTED</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>min_cost</th>
      <td>55802.000000</td>
      <td>50324.000000</td>
      <td>54581.000000</td>
      <td>49701.000000</td>
      <td>61922.000000</td>
      <td>54470.000000</td>
      <td>61712.000000</td>
      <td>52449.000000</td>
    </tr>
    <tr>
      <th>max_cost</th>
      <td>72309.000000</td>
      <td>60689.000000</td>
      <td>70773.000000</td>
      <td>60172.000000</td>
      <td>85766.000000</td>
      <td>69741.000000</td>
      <td>82440.000000</td>
      <td>68511.000000</td>
    </tr>
    <tr>
      <th>average_cost</th>
      <td>62880.750000</td>
      <td>54201.230000</td>
      <td>61111.105000</td>
      <td>53922.790000</td>
      <td>74009.900000</td>
      <td>59236.900000</td>
      <td>70794.630000</td>
      <td>58892.755000</td>
    </tr>
    <tr>
      <th>min_time</th>
      <td>3.497686</td>
      <td>1.907724</td>
      <td>0.408789</td>
      <td>0.614799</td>
      <td>2.271810</td>
      <td>1.450575</td>
      <td>0.527983</td>
      <td>0.596173</td>
    </tr>
    <tr>
      <th>max_time</th>
      <td>5.933967</td>
      <td>3.427369</td>
      <td>1.288425</td>
      <td>1.326740</td>
      <td>3.772325</td>
      <td>2.763644</td>
      <td>1.469050</td>
      <td>1.042160</td>
    </tr>
    <tr>
      <th>average_time</th>
      <td>4.453023</td>
      <td>2.512768</td>
      <td>0.786179</td>
      <td>0.941636</td>
      <td>2.915474</td>
      <td>2.001247</td>
      <td>0.818187</td>
      <td>0.757612</td>
    </tr>
  </tbody>
</table>
</div>

In [ ]:
for instance in (TspInstance("TSPA.csv"), TspInstance("TSPB.csv")):
    for intra_type in ("nodes", "edges"):
        for steepest in (True, False):
            for initial_solution_getter in (random_solution, weighted_regret):
                for column in columns:
                    instance.plot(experiments[-1][0][3], column)
               

In [10]:
import pandas as pd
df = pd.read_csv("results_exp3.csv", index_col=0)

# display separately for instance A and B
df_a = df.iloc[:, :8]
df_b = df.iloc[:, 8:]

# sort columns by min_cost
df_a = df_a.reindex(df_a.loc["min_cost"].sort_values().index, axis=1)
df_b = df_b.reindex(df_b.loc["min_cost"].sort_values().index, axis=1)

display(df_a)
display(df_b)

,A GREEDY NODES WEIGHTED,A STEEPEST NODES WEIGHTED,A STEEPEST NODES RANDOM,A GREEDY NODES RANDOM,A GREEDY EDGES WEIGHTED,A STEEPEST EDGES WEIGHTED,A GREEDY EDGES RANDOM,A STEEPEST EDGES RANDOM
min_cost,75599.000000,77166.000000,79543.000000,79716.000000,84341.000000,85296.000000,91560.000000,94636.000000
max_cost,91733.000000,90033.000000,98280.000000,94861.000000,101374.000000,103198.000000,111538.000000,122066.000000
average_cost,83129.250000,83777.340000,88041.030000,86056.665000,91750.625000,92531.985000,101610.135000,104366.425000
min_time,0.855200,1.994131,3.496069,0.556165,0.598760,1.449627,0.386965,2.082852
max_time,1.813898,5.577237,5.967110,1.843326,1.052440,3.010580,1.006590,3.411806
average_time,1.200964,2.841779,4.673874,1.135089,0.777606,1.989892,0.625281,2.733647


,B GREEDY NODES WEIGHTED,B STEEPEST NODES WEIGHTED,B GREEDY EDGES WEIGHTED,B STEEPEST EDGES WEIGHTED,B GREEDY NODES RANDOM,B STEEPEST NODES RANDOM,B GREEDY EDGES RANDOM,B STEEPEST EDGES RANDOM
min_cost,49701.000000,50324.000000,52449.000000,54470.000000,54581.000000,55802.000000,61712.000000,61922.000000
max_cost,60172.000000,60689.000000,68511.000000,69741.000000,70773.000000,72309.000000,82440.000000,85766.000000
average_cost,53922.790000,54201.230000,58892.755000,59236.900000,61111.105000,62880.750000,70794.630000,74009.900000
min_time,0.614799,1.907724,0.596173,1.450575,0.408789,3.497686,0.527983,2.271810
max_time,1.326740,3.427369,1.042160,2.763644,1.288425,5.933967,1.469050,3.772325
average_time,0.941636,2.512768,0.757612,2.001247,0.786179,4.453023,0.818187,2.915474


# Conlusions:
- Comparing new methods 
    - Greedy-Nodes-Weighted was returned the best solution for both instances (on avg and min)
    - Second best method was Steepest-Nodes-Weighted
    - The fastest execution was for methods Greedy-Nodes-Random and Greedy-Wedges-Random
- Comparing with meethods from the previous laboratories
    - Suprisingly, all new methods had worse results than previous labs methods
    - We spotted one mistake in weighted method, that could a little explain, why we didn't get at least as good as weighted method in previous laboratories
    - 